In [11]:
import arkouda as ak
import arachne as ar
import pandas as pd
import numpy as np
import time
import json

In [12]:
# NOTE: Make sure to change the server name to whatever is applicable in your environment. If running locally, then use only ak.connect().
ak.connect("n96", 5555)

connected to arkouda server tcp://*:5555


In [99]:
# These are just wedges, we do not need subgraph isomorphism for these.
src0 = [    1,     1]
dst0 = [10002, 10003]
connection_type0 = ['n', 'n']
color0 = ['b', 'b']

src1 = [    1,     1, 10003, 10003]
dst1 = [10002, 10003, 30004, 30005]
connection_type1 = ['n', 'n', 'n', 'n']
color1 = ['b', 'b', 'b', 'b']

src2 = [    1,     1, 40005]
dst2 = [10002, 10003,     1]
connection_type2 = ['n', 'n', 's']
color2 = ['b', 'b', 'b_r']

src3 = [    1,     1, 40005, 40005, 50008]
dst3 = [10002, 10003, 50008,     1, 10003]
connection_type3 = ['n', 'n', 'n', 's', 's']
color3 = ['b', 'b', 'r', 'b_r', 'b_r']

src4 = [1, 1, 10003, 10003, 60007]
dst4 = [10002, 10003, 30004, 30005, 1]
connection_type4 = ['n', 'n', 'n', 'n', 's']
color4 = ['b', 'b', 'b', 'b', 'b_r']

src5 = [1, 1, 10003, 10003, 60007, 60007, 70010]
dst5 = [10002, 10003, 30004, 30005, 70010, 1, 30005]
connection_type5 = ['n', 'n', 'n', 'n', 'n', 's', 's']
color5 = ['b', 'b', 'b', 'b', 'r', 'b_r',  'b_r']

src6 = [1, 1, 40005, 40005, 80009, 80009, 10003, 90010]
dst6 = [10002, 10003, 50006, 50007, 90010, 90011, 50006, 50007]
connection_type6 = ['n', 'n', 'n', 'n', 'n', 'n', 's', 's']
color6 = ['b', 'b', 'r', 'r', 'y', 'y', 'b_r', 'b_y']

src7 = [1, 10002, 40005, 60007, 80009]
dst7 = [10002, 20003, 1, 10002, 20003]
connection_type7 = ['n', 'n', 's', 's', 's']
color7 = ['b', 'b', 'b_r', 'b_y', 'r_y']

subgraphs_as_list_of_tuples = [ (src0, dst0, connection_type0, color0, "0"),
                                (src1, dst1, connection_type1, color1, "1"),
                                (src2, dst2, connection_type2, color2, "2"),
                                (src3, dst3, connection_type3, color3, "3"),
                                (src4, dst4, connection_type4, color4, "4"),
                                (src5, dst5, connection_type5, color5, "5"),
                                (src6, dst6, connection_type6, color6, "6"),
                                (src7, dst7, connection_type7, color7, "7")
                            ]

In [14]:
df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")
transformed_dataset = ak.DataFrame(df.to_dict(orient='list'))

/tmp/ipykernel_1669245/4272365571.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/scratch/users/oaa9/arkouda-njit/arachne/data/OL_dataset.csv")


In [52]:
print(len(df))

1187308


In [82]:
check_src = [770470833243998, 770470833243998, 5336902671469085, 5336902671469085, 5922641557884817]
check_dst = [670190479948102, 722316406166167, 770470833243998, 5922641557884817, 722316406166167]
pairs = list(zip(check_src, check_dst))
matching_rows = df[df[['src', 'dst']].apply(tuple, axis=1).isin(pairs)][['src', 'dst', 'n_id']].drop_duplicates(['src', 'dst'], keep='first')
print(matching_rows)

                     src               dst  \
46965    770470833243998   670190479948102   
46966    770470833243998   722316406166167   
605693  5336902671469085  5922641557884817   
673351  5922641557884817   722316406166167   
673354  5336902671469085   770470833243998   

                                         n_id  
46965                      720575940620519333  
46966                      720575940620519333  
605693                     720575940607407746  
673351  720575940607407746_720575940620519333  
673354  720575940607407746_720575940620519333  


In [74]:
g = ar.PropGraph()
g.load_edge_attributes(transformed_dataset, source_column="src", destination_column="dst", 
                            relationship_columns=["s_bef", "s_bef_x", "s_bef_y", "s_bef_z", "s_af", "s_af_x", 
                                                    "s_af_y", "s_af_z", 's_x', "s_y", "s_z", "s_distance", "d_bef",
                                                    "d_bef_x", "d_bef_y", "d_bef_z", "d_af", "d_af_x", "d_af_y", "d_af_z",
                                                    "d_x", "d_y", "d_z", "d_distance", "n_id", "connection_type"])

print(f"Built graph with {len(g)} vertices and {g.size()} edges")

Built graph with 667711 vertices and 962796 edges


In [96]:
check_src = ak.array([770470833243998, 770470833243998, 5336902671469085, 5336902671469085, 5922641557884817])
check_dst = ak.array([670190479948102, 722316406166167, 770470833243998, 5922641557884817, 722316406166167])
perm = ak.coargsort([check_src, check_dst])
check_src = check_src[perm]
check_dst = check_dst[perm]
print(check_src)
print(check_dst)
matching_rows = ak.find([ak.array(check_src),ak.array(check_dst)],[g.edge_attributes['src'],g.edge_attributes['dst']])
print(g.edge_attributes['n_id'][matching_rows])

[770470833243998 770470833243998 5336902671469085 5336902671469085 5922641557884817]
[670190479948102 722316406166167 770470833243998 5922641557884817 722316406166167]
['720575940620519333', '720575940620519333', '720575940607407746_720575940620519333', '720575940607407746', '720575940607407746_720575940620519333']


In [26]:
def get_mapping(g, subgraph, iso_cap):
    # Process subgraph information for mapping after subgraph isomorphism is invoked.
    src_sub, dst_sub = subgraph.edges()
    src_sub = src_sub.to_ndarray()
    dst_sub = dst_sub.to_ndarray()
    subgraph_nodes = sorted(list(np.unique(np.concatenate((src_sub, dst_sub)))))

    start = time.time()
    isos = ar.subgraph_isomorphism(g, subgraph, algorithm_type="si", return_isos_as="complete", semantic_check="or", size_limit=iso_cap)
    end = time.time()
    
    # Extract the returned array information from subgraph_isomorphism.
    isos_by_vertices = isos[0]
    isos_by_vertices_map = isos[1]
    isos_by_edges_src = isos[2]
    isos_by_edges_dst = isos[3]

    if len(isos_by_vertices) % len(subgraph) != 0:
        raise ValueError("The length of isomorphisms is not a multiple of the number of subgraph nodes.")

    # Get the number of motifs found.
    number_isos_found = len(isos_by_vertices) // len(subgraph_nodes)
    print(f"Finding {number_isos_found:_} motifs took: {end-start} seconds.")

    # Prepare the returned isomorphisms as a 2D array.
    start = time.time()
    isos_ndarray = isos_by_vertices.to_ndarray()
    hostgraph_nodes = isos_ndarray.reshape(-1, len(subgraph_nodes))
    end = time.time()
    print(f"Reshaping isomorphisms took: {end-start} seconds.")

    # Create all mappings at once using a list comprehension.
    start = time.time()
    all_mappings = [
        {int(k): int(v) for k, v in zip(subgraph_nodes, hostgraph_nodes[i])}
        for i in range(number_isos_found)
    ]
    end = time.time()
    print(f"Generating mappings took: {end-start} seconds.")

    return (all_mappings,(isos_by_edges_src,isos_by_edges_dst))

In [27]:
def motif_to_viss(src, dst, connection_type, iso_cap):    
    subgraph_dict = {
        "src": src,
        "dst": dst,
        "connection_type": connection_type
        }
    
    subgraph = ar.PropGraph()
    df = ak.DataFrame(subgraph_dict)
    subgraph.load_edge_attributes(df, source_column="src", destination_column="dst", 
                                relationship_columns=["connection_type"])
    
    node_mapping = get_mapping(g, subgraph, iso_cap)
    return node_mapping

In [44]:
send_to_file = False
all_mappings = []
for tup in subgraphs_as_list_of_tuples:
    subgraph_name = f"subgraph{tup[4]}"
    cap = 5
    print(f"Querying for {subgraph_name}...")
    curr_mapping = motif_to_viss(tup[0], tup[1], tup[2], cap)
    all_mappings.append(curr_mapping)
    print()

    if send_to_file:
        filename = subgraph_name + f"_motifs_capped_at_{cap}.json"
        with open(filename, "w") as f:
            json.dump(curr_mapping[0], f)

Querying for subgraph0...
Finding 38 motifs took: 0.6063122749328613 seconds.
Reshaping isomorphisms took: 0.021455049514770508 seconds.
Generating mappings took: 6.914138793945312e-05 seconds.

Querying for subgraph1...
Finding 21 motifs took: 1.4976074695587158 seconds.
Reshaping isomorphisms took: 0.02127671241760254 seconds.
Generating mappings took: 8.845329284667969e-05 seconds.

Querying for subgraph2...
Finding 8 motifs took: 0.5874297618865967 seconds.
Reshaping isomorphisms took: 0.021523237228393555 seconds.
Generating mappings took: 3.4809112548828125e-05 seconds.

Querying for subgraph3...
Finding 5 motifs took: 0.6271035671234131 seconds.
Reshaping isomorphisms took: 0.01333761215209961 seconds.
Generating mappings took: 3.1948089599609375e-05 seconds.

Querying for subgraph4...
Finding 7 motifs took: 0.6285593509674072 seconds.
Reshaping isomorphisms took: 0.02135324478149414 seconds.
Generating mappings took: 3.886222839355469e-05 seconds.

Querying for subgraph5...
Fin

In [ ]:
for tup in subgraphs_as_list_of_tuples:
    subgraph_id = int(tup[4])
    subgraph_name = f"subgraph{subgraph_id}"
    subgraph_src = tup[0]
    subgraph_dst = tup[1]
    color = tup[3]

    dict_to_check_against = {}
    for c in color:
        if c not in dict_to_check_against:
            dict_to_check_against[c] = 1
        else:
            dict_to_check_against[c] += 1

    print(f"Post processing {subgraph_name}...")

    isos_by_vertices = all_mappings[subgraph_id][0]
    isos_by_edges_src = all_mappings[subgraph_id][1][0]
    isos_by_edges_dst = all_mappings[subgraph_id][1][1]
    
    num_edges_subgraph = len(subgraph_src)
    number_isos_found = len(isos_by_edges_src) // len(subgraph_src)

    indices = ak.find([isos_by_edges_src,isos_by_edges_dst],[g.edge_attributes["src"], g.edge_attributes["dst"]])
    vals = g.edge_attributes["n_id"][indices]

    # TODO: Can the below be done with Arkouda?
    isos_by_edges_src_ndarray = np.split(isos_by_edges_src.to_ndarray(), number_isos_found)
    isos_by_edges_dst_ndarray = np.split(isos_by_edges_dst.to_ndarray(), number_isos_found)
    vals_ndarray = np.split(vals.to_ndarray(), number_isos_found)

    matches = 0
    curr_mapping_id = 0
    for src,dst,vals in zip(isos_by_edges_src_ndarray,isos_by_edges_dst_ndarray,vals_ndarray):
        if subgraph_id == 0 or subgraph_id == 1:
            if len(np.unique(vals)) == 1:
                matches += 1
                continue
        else:
            inner_matches = {}
            for val in vals:
                if val not in inner_matches:
                    inner_matches[val] = 1
                else:
                    inner_matches[val] += 1
            if sorted(inner_matches.values()) == sorted(dict_to_check_against.values()):
                matches += 1
            # else:
            #     print(isos_by_vertices[curr_mapping_id])
            #     print(src)
            #     print(dst)
            curr_mapping_id += 1

    print(f"Before post processing number of motifs found was {number_isos_found} and after was {matches}")
    print()


Post processing subgraph0...
Before post processing number of motifs found was 38 and after was 38

Post processing subgraph1...
Before post processing number of motifs found was 21 and after was 21

Post processing subgraph2...
Before post processing number of motifs found was 8 and after was 8

Post processing subgraph3...
Before post processing number of motifs found was 5 and after was 5

Post processing subgraph4...
Before post processing number of motifs found was 7 and after was 7

Post processing subgraph5...
Before post processing number of motifs found was 5 and after was 5

Post processing subgraph6...
{1: 1366038068497042, 10002: 1086128392124120, 10003: 1153149018861276, 40005: 130871608967581, 50006: 180367922615124, 50007: 90393691450449, 80009: 1081200404863751, 90010: 821002677531258, 90011: 954058333553553}
[1366038068497042 1366038068497042 1153149018861276  130871608967581
  130871608967581 1081200404863751 1081200404863751  821002677531258]
[1086128392124120 115314

In [106]:
print(all_mappings[3][0])

[{1: 298237460718741, 10002: 336872136431705, 10003: 278225254594359, 40005: 144859603770286, 50008: 264388566444025}, {1: 429343149753935, 10002: 324929389196380, 10003: 395058034346440, 40005: 351544664578740, 50008: 317712379381490}, {1: 638144388518750, 10002: 600301545007819, 10003: 492993755874517, 40005: 673342486984612, 50008: 539697112075013}, {1: 553629880263142, 10002: 497707670137828, 10003: 558492681146078, 40005: 972230658833868, 50008: 901624566055464}, {1: 770470833243998, 10002: 670190479948102, 10003: 722316406166167, 40005: 5336902671469085, 50008: 5922641557884817}]
